<a href="https://colab.research.google.com/github/rrfsantos/Desafios-NLP/blob/main/insert_label_iob_convert_spacy_arquivos_separados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python -m pip uninstall -y spacy==2.2.4
!python -m pip install pip setuptools wheel
!python -m pip install spacy[transformers,lookups]
!python -m spacy download en_core_web_sm  ### o colab não suportou executar a conversão com en_core_web_md e en_core_web_lg

Uninstalling spacy-3.0.6:
  Successfully uninstalled spacy-3.0.6
  Using cached https://files.pythonhosted.org/packages/1b/d8/0361bbaf7a1ff56b44dca04dace54c82d63dad7475b7d25ea1baefafafb2/spacy-3.0.6-cp37-cp37m-manylinux2014_x86_64.whl
2021-04-26 16:55:51.470250: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [1]:
import pandas as pd
import json
import warnings
import csv
import spacy
#import numpy as np
import re
from os import system

warnings.filterwarnings("ignore")

In [2]:
!python -m spacy info

2021-04-26 17:00:27.277802: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

============================== Info about spaCy ==============================

spaCy version    3.0.6                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.10                        
Pipelines        en_core_web_sm (3.0.0)        



In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
workdir_path = '/content/drive/My Drive/desafio 2/'
os.chdir(workdir_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
dirs = ['BC4CHEMD','BC5CDR-chem','s800','linnaeus','BC5CDR-disease','NCBI-disease','BC2GM','JNLPBA']

### Arquivos de treino

##### Ler arquivos IOB e incluir o nome da classe à tag

In [5]:
for dir in dirs:
    df = pd.read_csv('NERdata/' + dir + '/train_dev.tsv', sep='\t', header=None, names=['token','tag'],
                         quoting=csv.QUOTE_NONE, error_bad_lines=False, skipinitialspace = True)
       
    if (dir == 's800' or dir == 'linnaeus'):
      df['tag'] = df['tag'].replace(['B','I'],['B-SPECIES','I-SPECIES'])
    elif (dir == 'BC5CDR-disease' or dir == 'NCBI-disease'):
      df['tag'] = df['tag'].replace(['B','I'],['B-DISEASE','I-DISEASE'])
    elif (dir == 'BC2GM' or dir == 'JNLPBA'):
      df['tag'] = df['tag'].replace(['B','I'],['B-DRUG_PROTEIN','I-DRUG_PROTEIN'])
    else:
      df['tag'] = df['tag'].replace(['B','I'],['B-CHEMICALS','I-CHEMICALS'])
  
    df.dropna(inplace=True)

    df.to_csv('IOBdata/iob_train_' + dir + '.iob', sep='\t', index=False, header=False)

##### Converter arquivos em formato IOB para o formato binário .spacy

In [6]:
for dir in dirs:  
  iob_file = 'IOBdata/iob_train_' + dir + '.iob'
  folder_path = './SPACYdata/train'
      
  system(f'python -m spacy convert -c ner -s -n 10 -b en_core_web_sm {iob_file} {folder_path}') #v3  

### Arquivos de Teste

##### Ler arquivos IOB e incluir o nome da classe à tag

In [8]:
for dir in dirs:
    df = pd.read_csv('NERdata/' + dir + '/test.tsv', sep='\t', header=None, names=['token','tag'],
                         quoting=csv.QUOTE_NONE, error_bad_lines=False, skipinitialspace = True)
     

    if (dir == 's800' or dir == 'linnaeus'):
      df['tag'] = df['tag'].replace(['B','I'],['B-SPECIES','I-SPECIES'])
    elif (dir == 'BC5CDR-disease' or dir == 'NCBI-disease'):
      df['tag'] = df['tag'].replace(['B','I'],['B-DISEASE','I-DISEASE'])
    elif (dir == 'BC2GM' or dir == 'JNLPBA'):
      df['tag'] = df['tag'].replace(['B','I'],['B-DRUG_PROTEIN','I-DRUG_PROTEIN'])
    else:
      df['tag'] = df['tag'].replace(['B','I'],['B-CHEMICALS','I-CHEMICALS'])

    df.dropna(inplace=True)

    df.to_csv('IOBdata/iob_test_' + dir + '.iob', sep='\t', index=False, header=False)  

##### Converter arquivos em formato IOB para o formato binário .spacy

In [9]:
for dir in dirs:  
  iob_file = 'IOBdata/iob_test_' + dir + '.iob'
  folder_path = './SPACYdata/test'
      
  system(f'python -m spacy convert -c ner -s -n 10 -b en_core_web_sm {iob_file} {folder_path}') #v3  